<a href="https://colab.research.google.com/github/aronnyberg/finance-scripts/blob/master/dowloadBasks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#necessary imports, on colab you need to !pip install some of the less popular 
#ones
!pip install ccxt
import ccxt
from datetime import datetime

!pip install --upgrade -q gspread

import gspread
import pandas as pd
from google.colab import auth
from oauth2client.client import GoogleCredentials
import time

from google.colab import drive
drive.mount('drive')

auth.authenticate_user()
gc = gspread.authorize(GoogleCredentials.get_application_default())

     |████████████████████████████████| 2.2 MB 8.8 MB/s 
     |████████████████████████████████| 1.1 MB 44.4 MB/s 
     |████████████████████████████████| 3.6 MB 39.5 MB/s 
     |████████████████████████████████| 271 kB 44.9 MB/s 
     |████████████████████████████████| 160 kB 43.7 MB/s 
     |████████████████████████████████| 291 kB 45.6 MB/s 
     |████████████████████████████████| 192 kB 45.8 MB/s 
Mounted at drive


In [ ]:
#load up all the exchanges you want to grab data, their names will be linked to 
#later on
binance = ccxt.binance () 
binance.load_markets ()
ftx = ccxt.ftx () 
ftx.load_markets ()
gemini = ccxt.gemini ()
gemini.load_markets ()
kraken = ccxt.kraken () 
kraken.load_markets ()
bitfinex = ccxt.bitfinex ()
bitfinex.load_markets ()
bitstamp = ccxt.bitstamp () 
bitstamp.load_markets ()
bitmart = ccxt.bitmart()
bitmart.load_markets()
gateio = ccxt.gateio()
gateio.load_markets()
phemex = ccxt.phemex()
phemex.load_markets()

{'AAVE/USD:USD': {'active': True,
  'base': 'AAVE',
  'baseId': 'AAVE',
  'contractSize': '0.01',
  'id': 'AAVEUSD',
  'info': {'contractSize': '0.01',
   'contractUnderlyingAssets': 'AAVE',
   'displaySymbol': 'AAVE / USD',
   'fundingRate8hSymbol': '.AAVEFR8H',
   'fundingRateSymbol': '.AAVEFR',
   'indexSymbol': '.AAVE',
   'lotSize': '1',
   'markSymbol': '.MAAVE',
   'maxOrderQty': '200000',
   'maxPriceEp': '1000000000',
   'minPriceEp': '100',
   'pricePrecision': '2',
   'priceScale': '4',
   'quoteCurrency': 'USD',
   'ratioScale': '8',
   'riskLimits': [{'initialMargin': '5.0%',
     'initialMarginEr': '5000000',
     'limit': '500000',
     'maintenanceMargin': '1.0%',
     'maintenanceMarginEr': '1000000'},
    {'initialMargin': '5.5%',
     'initialMarginEr': '5500000',
     'limit': '1000000',
     'maintenanceMargin': '1.5%',
     'maintenanceMarginEr': '1500000'},
    {'initialMargin': '6.0%',
     'initialMarginEr': '6000000',
     'limit': '1500000',
     'maintenance

In [ ]:
#function that appends data, nicely formatted on top of each other as a flat database
def get_basks(thepair, theexchange):
  basks = theexchange.fetch_ticker(thepair)
  bask_dict['bid'].append(basks['bid'])
  bask_dict['ask'].append(basks['ask'])
  bask_dict['asset'].append(basks['symbol'])
  bask_dict['exchange'].append(str(theexchange).upper())

#Below is just a list of the top 200 cryptos by market cap I pulled from some website
#The code below it just sorts the token tickers into a list
wb = gc.open_by_url('https://docs.google.com/spreadsheets/d/1WcKX9sf6e5WujxYs9mUJVMeJ5beYgDglJShzl2T5JeM/edit#gid=0')
data = wb.worksheet('Sheet1').get_all_values()
assets = pd.DataFrame(data)
assetList = assets.iloc[:,2].values
assetList = [i for i in assetList]

In [ ]:
#Grabs data for all available 50th to 200th biggest market cap cryptos, sorting into that flat database
#Why not the top 50? A hunch on competition
#basks = bid, asks
bask_dict = {'bid':[], 'ask':[], 'asset':[], 'exchange':[]}
for each in assetList[50:]:
  for base in ['USD', 'GBP', 'EUR', 'JPY', 'ETH', 'USDT', 'USDC']:
    ticker = each+'/'+base
    for exch in [binance, ftx, kraken, bitfinex, bitmart, gateio, phemex]:
      try:
        get_basks(ticker, exch)
      except:
        pass
      time.sleep(0.25)

In [ ]:
#Save your data with todays date to your google drive (when using colab)
pd.DataFrame(bask_dict).to_csv('basks301221.csv')
!cp basks301221.csv "drive/My Drive/"